In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from datasets import load_dataset
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from tqdm.auto import tqdm


import ast
import os
import pandas as pd

In [6]:
from dotenv import load_dotenv
import os
env_path = r'E:\YTReusable\.env'
load_dotenv(env_path)
# Access variables from the .env file
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [8]:
openai_client = OpenAI()
pinecone = Pinecone()

In [10]:
# Create or connect to Pinecone index
index_name = 'rag-example8'
if index_name not in pinecone.list_indexes():
   pinecone.create_index(name=index_name, dimension=1536, metric='cosine',spec=ServerlessSpec(cloud='aws', region='us-west-2'))

In [12]:
index = pinecone.Index(index_name)

In [14]:
# Sample Data: A small set of documents or FAQs (can be imported easily)
data = {
    "questions": [
        "What is AI?",
        "What is Pinecone?",
        "How do embeddings work?",
        "What is GPT?",
        "What is machine learning?"
    ],
    "answers": [
        "AI is the simulation of human intelligence in machines.",
        "Pinecone is a vector database that enables efficient search and retrieval.",
        "Embeddings are a numerical representation of text that captures semantic meaning.",
        "GPT is a transformer-based model that generates human-like text.",
        "Machine learning is a subset of AI where machines learn from data to make decisions."
    ]
}
df = pd.DataFrame(data)

In [16]:
df.head()

,questions,answers
0,What is AI?,AI is the simulation of human intelligence in ...
1,What is Pinecone?,Pinecone is a vector database that enables eff...
2,How do embeddings work?,Embeddings are a numerical representation of t...
3,What is GPT?,GPT is a transformer-based model that generate...
4,What is machine learning?,Machine learning is a subset of AI where machi...


In [18]:
# Function to get OpenAI embeddings
def get_embeddings(text):
    response = openai_client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return response.data[0].embedding  # Correctly access the embedding

In [20]:
# Insert data into Pinecone
def insert_data_to_pinecone(df):
    vectors = []
    for idx, row in df.iterrows():
        # Get the embedding for the answer text using OpenAI API
        response = openai_client.embeddings.create(input=[row['answers']], model="text-embedding-ada-002")
        embedding = response.data[0].embedding
        
        
        embedding_list = list(embedding)  # Explicitly convert to a list
        #print(embedding_list)
        vectors.append((str(idx), embedding_list, {'text': row['answers']}))  
    
    # Upsert data to Pinecone
    index.upsert(vectors=vectors)


In [24]:
insert_data_to_pinecone(df)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

In [26]:
# Function to retrieve the most similar document from Pinecone
def retrieve_relevant_documents(query, top_k=3):
    query_embedding = get_embeddings(query)
    
    # Ensure the query embedding is a list of floats (not ndarray)
    query_embedding = list(query_embedding)  # Convert to list if necessary
    
    # Query Pinecone using the correct keyword arguments
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    return results['matches']


In [28]:
# Function to generate a rich response using GPT-3 based on retrieved documents
def generate_answer(query):
    # Step 1: Retrieve relevant documents from Pinecone
    matches = retrieve_relevant_documents(query)
    
    
    context = "\n".join([match['metadata'].get('text', 'No text found') for match in matches])
    
    
    
    messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Here are some documents related to your question:\n{context}\n\nQuestion: {query}"},
    ]
    print(messages)
    # Step 4: Use GPT-3.5-turbo to generate a response based on the context and question
    response = openai_client.chat.completions.create(
        model="gpt-4-turbo",  
        messages=messages,      
        max_tokens=500,
        temperature=0.7        
    )
    
    return response.choices[0].message.content

In [34]:
# Test the RAG system
query = "What is machine learning - please explain in 300 words"
answer = generate_answer(query)
print(f"Question: {query}\nAnswer: {answer}")

[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Here are some documents related to your question:\nMachine learning is a subset of AI where machines learn from data to make decisions.\nAI is the simulation of human intelligence in machines.\nGPT is a transformer-based model that generates human-like text.\n\nQuestion: What is machine learning - please explain in 300 words'}]
Question: What is machine learning - please explain in 300 words
Answer: Machine learning (ML) is a branch of artificial intelligence (AI) focused on building systems that learn and improve from experience without being explicitly programmed. The essence of machine learning revolves around developing algorithms that can process input data and use statistical analysis to predict an output while updating outputs as new data becomes available. The processes involve recognizing patterns in data and making decisions with minimal human intervention.

The foundation of machine 